# Defining chemical systems

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 4th, 2022</p>

Before performing chemical reaction calculations, such as chemical equilibrium and kinetics, we need to determine the phases that must be considered in the calculation and the chemical species that constitute these phases.

For example, to calculate the solubility of a mineral in water, two phases must be considered:

* an aqueous phase representing water and dissolved species; and
* a solid phase representing the mineral we want to dissolve in the aqueous solution.

Another example is the combustion of a solid substance. We must define a chemical system composed of:

* a gaseous phase containing most of the gases relevant to the process; and
* one or more solid/liquid substances (condensed phases) that may appear during the combustion process.

The concept of a chemical system is represented in Reaktoro using the {{ChemicalSystem}} class. You should expect to construct one object of this class for almost every chemical process you want to model. 

In a {{ChemicalSystem}} object, you can find a list of {{Phase}} objects representing the phases. The {{Phase}} class is also an essential component of Reaktoro. It contains the list of chemical species composing the phase, as a list of {{Species}} objects. Additionally, each {{Phase}} object includes its activity model to account for its non-ideal thermodynamic behavior. 

```{note}
We'll learn more about activity models in a subsequent section and how to specify them for our phases.
```

Next, we will present how to construct a chemical system in Reaktoro for different applications.

## Chemical system definition for a mineral solubility problem

Let's consider the chemical equilibrium problem associated with calculating the solubility of halite (NaCl) in water. The code block below demonstrates how a {{ChemicalSystem}} object is created with an aqueous and mineral phase.

In [1]:
from reaktoro import *

# Let's use one of PHREEQC's database
db = PhreeqcDatabase("phreeqc.dat")

# Define an aqueous solution with species that are pertinent to the problem
solution = AqueousPhase("H2O H+ OH- Na+ Cl-")

# Because we want to compute the solubility of halite, a mineral phase is needed
halite = MineralPhase("Halite")

# We can now create our ChemicalSystem object with the above phase specifications
system = ChemicalSystem(db, solution, halite)

That's it! With the {{ChemicalSystem}} object created above, we can do some exciting things. For example, we can inspect the phases in the system and their composing chemical species:

In [2]:
for phase in system.phases():
    print(phase.name())
    for species in phase.species():
        print(":: " + species.name())

AqueousPhase
:: H2O
:: H+
:: OH-
:: Na+
:: Cl-
Halite
:: Halite


```{note}
Reaktoro has default names for phases, such as `AqueousPhase` and `GaseousPhase`` for aqueous and gaseous phases. For pure mineral phases, the phase name is the same as the name of the mineral species composing it. That's is why the mineral phase above is named `Halite`, whose only composing species is called `Halite`. Note also that Reaktoro supports as many phases as you wish, each phase containing any number of species. We will demonstrate this in subsequent tutorials.
```

We can also inspect how the chemical species are ordered in the system:

In [3]:
for species in system.species():
    print(species.name())

H2O
H+
OH-
Na+
Cl-
Halite


And inspect the chemical elements in the system:

In [4]:
for element in system.elements():
    print(element.symbol())

H
O
Na
Cl


The {{ChemicalSystem}} class also contains the formula matrix $A$ of the system, which is a matrix whose entry $A_{j,i}$ contains the number of atoms of element with index $j$ in species with index $i$:

In [5]:
print(system.formulaMatrix())

[[ 2.  1.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.]
 [ 0.  1. -1.  1. -1.  0.]]


The last row in the formula matrix contains the electric charge of each species.

```{tip}
Access the link {{ChemicalSystem}} to find out all methods available in class `ChemicalSystem`!
```

## Chemical system definition for a gas solubility problem

We now consider a chemical system of interest for the computation of CO<sub>2</sub> solubility in saline solutions. For this case, we will need an aqueous and gaseous phase:

In [1]:
from reaktoro import *

# Let's use the SUPCRTBL database this time
db = SupcrtDatabase("supcrtbl")

# Define an aqueous solution with automatic species collection for given selected elements
solution = AqueousPhase(speciate("H O Na Cl Ca C"))

# Let Reaktoro automatically identify the gases by specifying an empty list of species below
gas = GaseousPhase()

# We can now create our ChemicalSystem object with the above phase specifications
system = ChemicalSystem(db, solution, gas)

Let's print the species in the system, their names, formula, and molar mass:

In [7]:
print("{:<25}{:<25}{:<20}".format("Name", "Formula", "Molar Mass (kg/mol)"))
for species in system.species():
    print("{:<25}{:<25}{:<20.6f}".format(species.name(), species.formula().str(), species.molarMass()))

Name                     Formula                  Molar Mass (kg/mol) 
H2O(aq)                  H2O                      0.018015            
CaOH+                    CaOH+                    0.057085            
CO(aq)                   CO                       0.028010            
CO2(aq)                  CO2                      0.044010            
CO3-2                    CO3-2                    0.060010            
Ca(HCO3)+                Ca(HCO3)+                0.101095            
Ca+2                     Ca+2                     0.040077            
CaCl+                    CaCl+                    0.075530            
CaCl2(aq)                CaCl2                    0.110983            
Cl-                      Cl-                      0.035453            
HClO(aq)                 HClO                     0.052460            
ClO-                     ClO-                     0.051453            
ClO2-                    ClO2-                    0.067452            
ClO3- 

This chemical system contains many species. The SUPCRTBL database contains many
organic species that should play an insignificant role in accurately computing
CO{{_2}} solubility. All organic species in SUPCRT and SUPCRTBL databases in
Reaktoro’s YAML format have an organic tag which we can use to filter out them.

Let's then redefine our {{ChemicalSystem}} object `system` by excluding those
organic aqueous species and specifying exactly the gases we want:

In [8]:
solution = AqueousPhase(speciate("H O Na Cl Ca C"), exclude("organic"))

gas = GaseousPhase("CO2(g)")

system = ChemicalSystem(db, solution, gas)

```{warning}
Make sure you specify species names exactly how they exist in the {{Database}}
object you have created, such as in `GaseousPhase("CO2(g)")` above. Otherwise
you will get a runtime exception!
```

And here is the updated list of species (only their names this time):

In [9]:
for species in system.species():
    print(species.name())

H2O(aq)
CaOH+
CO(aq)
CO2(aq)
CO3-2
Ca(HCO3)+
Ca+2
CaCl+
CaCl2(aq)
Cl-
HClO(aq)
ClO-
ClO2-
ClO3-
ClO4-
H+
H2(aq)
HCO3-
HO2-
Na+
NaCl(aq)
NaOH(aq)
O2(aq)
OH-
H2O2(aq)
HClO2(aq)
HCl(aq)
CaCO3(aq)
CO2(g)


You may still be discontent with so many aqueous species for the sake of CO<sub>2</sub> solubility calculation. We demonstrate with the code block below how to specify the exact aqueous species to be considered in the phase using a list of species names.

In [10]:
aqueous_species = [
    "H2O(aq)", 
    "CaOH+", 
    "CO2(aq)", 
    "CO3-2", 
    "Ca(HCO3)+", 
    "Ca+2", 
    "CaCl+", 
    "Cl-", 
    "H+", 
    "HCO3-", 
    "HO2-", 
    "Na+", 
    "OH-"
]

solution = AqueousPhase(aqueous_species)

system = ChemicalSystem(db, solution, gas)

In [11]:
for species in system.species():
    print(species.name())

H2O(aq)
CaOH+
CO2(aq)
CO3-2
Ca(HCO3)+
Ca+2
CaCl+
Cl-
H+
HCO3-
HO2-
Na+
OH-
CO2(g)


## Chemical system definition with many phases

Now, let's overcomplicate and define a chemical system with many phases and
species. We have an aqueous solution for which we know the chemical elements of
interest. We want Reaktoro to automatically include in our chemical system all
minerals that could potentially be important (e.g., minerals that could
precipitate as the solution temperature is changed). These are minerals in the
database whose constituting elements are found in the aqueous solution. We also
want to show how a mineral phase can be defined as a solid solution (containing
more than one mineral end-member).

We demonstrate the above requirements for creating a chemical system in the
code block below. Note the use of the `cemdata18` database provided by
ThermoFun, which is suitable for modeling cement chemistry.

In [12]:
db = ThermoFunDatabase("cemdata18")

solution = AqueousPhase(speciate("H O Na Cl Ca Mg C Si Fe Al K S"))
gas = GaseousPhase()
pureminerals = MineralPhases()
solidsolution = MineralPhase("ettringite Fe-ettringite")

system = ChemicalSystem(db, solution, gas, pureminerals, solidsolution)

Let's now print the phase names and their composing species:

In [13]:
for phase in system.phases():
    print(phase.name())
    for species in phase.species():
        print(":: " + species.name())

AqueousPhase
:: Al(SO4)+
:: Al(SO4)2-
:: Al+3
:: AlO+
:: AlO2-
:: AlO2H@
:: AlOH+2
:: AlSiO5-3
:: CH4@
:: CO2@
:: CO3-2
:: Ca(CO3)@
:: Ca(HCO3)+
:: Ca(HSiO3)+
:: Ca(SO4)@
:: Ca+2
:: CaOH+
:: CaSiO3@
:: Cl-
:: ClO4-
:: Fe(CO3)@
:: Fe(HCO3)+
:: Fe(HSO4)+
:: Fe(HSO4)+2
:: Fe(SO4)+
:: Fe(SO4)2-
:: Fe(SO4)@
:: Fe+2
:: Fe+3
:: FeCl+
:: FeCl+2
:: FeCl2+
:: FeCl3@
:: FeO+
:: FeO2-
:: FeO2H@
:: FeOH+
:: FeOH+2
:: H+
:: H2@
:: H2O@
:: H2S@
:: HCO3-
:: HS-
:: HSO3-
:: HSO4-
:: HSiO3-
:: K(SO4)-
:: K+
:: KOH@
:: Mg(CO3)@
:: Mg(HCO3)+
:: Mg(HSiO3)+
:: Mg+2
:: MgOH+
:: MgSO4@
:: Na(CO3)-
:: Na(HCO3)@
:: Na(SO4)-
:: Na+
:: NaOH@
:: O2@
:: OH-
:: S2O3-2
:: SO3-2
:: SO4-2
:: Si4O10-4
:: SiO2@
:: SiO3-2
GaseousPhase
:: CH4
:: CO2
:: H2
:: H2O
:: H2S
:: O2
ettringite
:: ettringite
:: Fe-ettringite
5CA
:: 5CA
5CNA
:: 5CNA
AlOHam
:: AlOHam
AlOHmic
:: AlOHmic
Amor-Sl
:: Amor-Sl
Anh
:: Anh
Arg
:: Arg
Brc
:: Brc
C12A7
:: C12A7
C2AClH5
:: C2AClH5
C2AH65
:: C2AH65
C2AH7.5
:: C2AH7.5
C2S
:: C2S
C3A
:: C3A
C3AFS0

```{tip}
If efficient calculations are required in your application, you may want to be more selective in the phases and species that populate your chemical system! Unfortunately, it is not always possible to know in advance the species that do not make sense for our model. So do it carefully. Ensure that for all thermodynamic/chemical conditions of interest (e.g. for temperature, pressure ranges of interest) the species you want to exclude from the system are negligible (i.e. exist with numerically zero or tiny amounts).
```

Imagine, however, you are dealing with two minerals and water and you don't
want to specify the chemical elements in the definition of the aqueous phase.
This is what you can do:

In [14]:
db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase()
albite = MineralPhase("Albite")
kaolinite = MineralPhase("Kaolinite")

system = ChemicalSystem(db, albite, kaolinite, solution)

Let's now find out which aqueous species were selected automatically for our
aqueous phase:

In [15]:
aqueousphase = system.phases().get("AqueousPhase")
for species in aqueousphase.species():
    print(species.name())

H+
H2O
Al+3
Al(OH)2+
Al(OH)3
Al(OH)4-
AlOH+2
H2
H4SiO4
H2SiO4-2
H3SiO4-
Na+
OH-
NaOH
O2


## Chemical system definition for a combustion problem

Forget about water now and let's construct a chemical system suitable for modeling the combustion of black powder. Black powder is composed of potassium nitrate (KNO<sub>3</sub>), charcoal (C<sub>10</sub>Ca<sub>0.026</sub>H<sub>4.774</sub>N<sub>0.039</sub>O<sub>1.234</sub>), and sulfur (S<sub>8</sub>).

In the code block below we construct a chemical system using the chemical elements above. 


In [2]:
# Let's use the NASA-CEA database for this example
db = NasaDatabase("nasa-cea")

# Consider all possible condensed phases (solid or liquid substances) with given elements
condensed = CondensedPhases(speciate("K N O C Ca H S"))

# Automatically select the gases based on the elements above
gases = GaseousPhase()  

# Create a chemical system suitable for modeling combustion of black powder!
system = ChemicalSystem(db, gases, condensed)

We print below the gases and condensed phases constituting our chemical system:

In [3]:
print("Gases")
for species in system.species():
    if species.aggregateState() == AggregateState.Gas:
        print(":: " + species.name())
print("Condensed Phases")
for species in system.species():
    if species.aggregateState() == AggregateState.CondensedPhase:
        print(":: " + species.name())

Gases
:: e-
:: C
:: C+
:: C-
:: CH
:: CH+
:: CH2
:: CH3
:: CH2OH
:: CH2OH+
:: CH3O
:: CH4
:: CH3OH
:: CH3OOH
:: CN
:: CN+
:: CN-
:: CNN
:: CO
:: CO+
:: COS
:: CO2
:: CO2+
:: COOH
:: CS
:: CS2
:: C2
:: C2+
:: C2-
:: C2H
:: C2H2,acetylene
:: C2H2,vinylidene
:: CH2CO,ketene
:: O(CH)2O
:: HO(CO)2OH
:: C2H3,vinyl
:: CH3CN
:: CH3CO,acetyl
:: C2H4
:: C2H4O,ethylen-o
:: CH3CHO,ethanal
:: CH3COOH
:: OHCH2COOH
:: C2H5
:: C2H6
:: CH3N2CH3
:: C2H5OH
:: CH3OCH3
:: CH3O2CH3
:: CCN
:: CNC
:: OCCN
:: C2N2
:: C2O
:: C2S2
:: C3
:: C3H3,1-propynl
:: C3H3,2-propynl
:: C3H4,allene
:: C3H4,propyne
:: C3H4,cyclo-
:: C3H5,allyl
:: C3H6,propylene
:: C3H6,cyclo-
:: C3H6O,propylox
:: C3H6O,acetone
:: C3H6O,propanal
:: C3H7,n-propyl
:: C3H7,i-propyl
:: C3H8
:: C3H8O,1propanol
:: C3H8O,2propanol
:: CNCOCN
:: C3OS
:: C3O2
:: C3S2
:: C4
:: C4H2,butadiyne
:: C4H4,1,3-cyclo-
:: C4H6,butadiene
:: C4H6,1butyne
:: C4H6,2butyne
:: C4H6,cyclo-
:: C4H8,1-butene
:: C4H8,cis2-buten
:: C4H8,tr2-butene
:: C4H8,isobutene
:: C4H8

Let's check how many phases and species were collected in our system:

In [5]:
print("Number of species in the system:", system.species().size())
print("Number of phases in the system:", system.phases().size())

Number of species in the system: 317
Number of phases in the system: 67


This is an impressive number of species and phases to be able to model the combustion of black powder! 

```{note}
By including as many gases and condensed phases as possible, Reaktoro will be able to determine those that may appear after burning black powder. For example, K{{_2}}S(cd), K{{_2}}SO{{_4}}(cd) and CaS(cd) are typically formed in the combustion process. By including them in the definition of the chemical system, the chemical equilibrium solver will be able to find positive amounts for them (i.e., the solver will identify these condensed phases as stable in equilibrium).
```

Keep reading to learn more about how the {{ChemicalSystem}} class plays an important role in Reaktoro!